### DNN para a Sinalização Brasileria

Deep Neural Network ou DNN é um módulo do OpenCV, trata-se de um módulo para realizar a infêrncias através de redes neurais pré-treinadas de imagens e videos. Tenha em mente que não é possivel realizar treinamento usando esse módulo. Este código foi baseado no repositório dnn_opencv de ArnaldoJR, disponível no github em: https://github.com/arnaldojr/dnn_opencv

In [1]:
import os
import cv2
print("OpenCV version:", cv2.__version__)
import numpy as np

OpenCV version: 4.5.5


In [2]:
class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))
print(class_names)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']


In [4]:
def draw_boxes_masks(image, boxes, masks, labels, confidence=0.5, threshold=0.3):
    clone = image.copy()
    rois = []
    for i in range(0, boxes.shape[2]):
        classID = int(boxes[0, 0, i, 1])
        score = boxes[0, 0, i, 2]
        if score > confidence:
            (H, W) = clone.shape[:2]
            box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])

            (startX, startY, endX, endY) = box.astype("int")
            boxW = endX - startX
            boxH = endY - startY

            mask = masks[i, classID]
            mask = cv2.resize(mask, (boxW, boxH),interpolation=cv2.INTER_CUBIC)
            mask = (mask > threshold)

            roi = clone[startY:endY, startX:endX]

            roi = roi[mask]

            color = COLORS[classID]

            blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")

            clone[startY:endY, startX:endX][mask] = blended

            color = [int(c) for c in color]
            cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)

            text = "{}: {:.4f}".format(labels[classID], score)
            cv2.putText(clone, text, (startX, startY - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return clone

referencia da mask rcnn coco: https://github.com/sambhav37/Mask-R-CNN

In [3]:
# load the DNN model
model = cv2.dnn.readNet(model='Mask-R-CNN/mask-rcnn-coco/frozen_inference_graph.pb',
                        config='Mask-R-CNN/mask-rcnn-coco/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt',framework='TensorFlow')

In [15]:


# import the necessary packages

import cv2
import numpy as np
import time

class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '']
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Mask-R-CNN/mask-rcnn-coco/frozen_inference_graph.pb',
                        config='Mask-R-CNN/mask-rcnn-coco/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt',framework='TensorFlow')


image_x, image_y = 32,32


#classifier = load_model("LeNet5_BRA.model")
    
cam = cv2.VideoCapture(0)

img_counter = 0

img_text = ['','']
while True:
    ret, frame = cam.read()

    blob = cv2.dnn.blobFromImage(image=frame, swapRB=True, crop=False)
    # set the blob to the model
    model.setInput(blob)
    
    output = model.forward(["detection_out_final", "detection_masks"])
    (boxes, masks) = output

    drawed_image = draw_boxes_masks(
        frame,
        boxes,
        masks,
        class_names
    )

    cv2.imshow('image', drawed_image)
       
        
    if cv2.waitKey(1) == 113:
        break

cam.release()
cv2.destroyAllWindows()